# Text Classification
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement a of spam classifier with **Naive Bayes method** for real world email messages
- learn the **training and testing phase** for Naive Bayes classifier  
- get an idea of the **precision-recall** tradeoff

In [2]:
# some basic imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
# ham_train contains the occurrences of each word in ham emails. 1-by-N vector
ham_train = np.loadtxt('ham_train.csv', delimiter=',')
# spam_train contains the occurrences of each word in spam emails. 1-by-N vector
spam_train = np.loadtxt('spam_train.csv', delimiter=',')
# N is the size of vocabulary.
print ham_train.shape
N = ham_train.shape[0]
# There 9034 ham emails and 3372 spam emails in the training samples
num_ham_train = 9034
num_spam_train = 3372
# Do smoothing
x = np.vstack([ham_train, spam_train]) + 1

# ham_test contains the occurences of each word in each ham test email. P-by-N vector, with P is number of ham test emails.
i,j,ham_test = np.loadtxt('ham_test.txt').T 
i = i.astype(np.int)
j = j.astype(np.int)
ham_test_tight = scipy.sparse.coo_matrix((ham_test, (i - 1, j - 1)))
ham_test = scipy.sparse.csr_matrix((ham_test_tight.shape[0], ham_train.shape[0]))
ham_test[:, 0:ham_test_tight.shape[1]] = ham_test_tight
# spam_test contains the occurences of each word in each spam test email. Q-by-N vector, with Q is number of spam test emails.
i,j,spam_test = np.loadtxt('spam_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
spam_test_tight = scipy.sparse.csr_matrix((spam_test, (i - 1, j - 1)))
spam_test = scipy.sparse.csr_matrix((spam_test_tight.shape[0], spam_train.shape[0]))
spam_test[:, 0:spam_test_tight.shape[1]] = spam_test_tight


(77386,)


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:20: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:27: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


## Now let's implement a ham/spam email classifier. Please refer to the PDF file for details

In [4]:
from likelihood import likelihood
# TODO
# Implement a ham/spam email classifier, and calculate the accuracy of your classifier

# Hint: you can directly do matrix multiply between scipy.sparse.coo_matrix and numpy.array.
# Specifically, you can use sparse_matrix * np_array to do this. Note that when you use "*" operator
# between numpy array, this is typically an elementwise multiply.

# begin answer
L = likelihood(x)
prior = np.zeros([2],dtype='float')
prior[0]= (9034.0+1)/(9034+3372+2)  # P(ham)
prior[1] = (3372.0+1)/(9034+3372+2) # P(spam)
# top 10 words
f = open("all_word_map.txt","r")
word_map = f.readlines()

words = np.zeros([N])
words = L[1]/L[0]
words_sort = np.argsort(-words)
print "Top 10 spam words: "
for i in range(10):
    t = word_map[words_sort[i]].find('\t')
    print words_sort[i], word_map[words_sort[i]][:t]
    
    


Top 10 spam words: 
30032 nbsp
75525 viagra
38175 pills
45152 cialis
9493 voip
65397 php
37567 meds
13612 computron
56929 sex
9452 ooking


In [21]:
log_prior = np.log(prior)
log_L = np.log(L)

post = np.zeros([2,N])
for i in range(2):
    post[i] = log_L[i]+log_prior[i]
    
fp = 0
for i in range(ham_test.shape[0]):
    if (ham_test[i]*log_L[0])+log_prior[0] < (ham_test[i]*log_L[1])+log_prior[1]:
        fp += 1
print "fp:",fp
print "tn:",ham_test.shape[0]-fp
fn = 0    
for i in range(spam_test.shape[0]):
    if (spam_test[i]*log_L[0])+log_prior[0] > (spam_test[i]*log_L[1])+log_prior[1]:
        fn += 1
print "fn:",fn
print "tp:",spam_test.shape[0]-fn
print "accuracy: ",1-(fn+fp*1.0)/(spam_test.shape[0]+ham_test.shape[0])
        
# end answer

fp: 31
tn: 2980
fn: 27
tp: 1097
accuracy:  0.985973397823
